In [5]:
import os
import cv2
import numpy as np
import pandas as pd
import ast
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
import re
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df=pd.read_csv('sentiment.csv')
df

,Unnamed: 0,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw,clean_tokens
0,0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.,"['plate', 'delicious', 'food', 'including', 'f..."
1,1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...,"['french', 'fry', 'healthy', 'food', 'excellen..."
2,2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ...","['plate', 'one', 'favorite', 'food', 'french',..."
3,3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food.","['disgusting', 'food', 'bad', 'food']"
4,4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.,"['plate', 'disgusting', 'food', 'found', 'diner']"
...,...,...,...,...,...,...,...,...,...,...
39194,39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...,"['dirty', 'bathroom', 'dirty', 'window', 'made..."
39195,39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.,"['dirty', 'bathroom', 'window']"
39196,39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom,"['towel', 'rack', 'dirty', 'bathroom']"
39197,39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...,"['dirty', 'bathroom', 'dirty', 'window', 'made..."


In [3]:
df['tokens'][0]

"['a', 'plate', 'of', 'delicious', 'food', 'including', 'french', 'fries']"

In [6]:
my_list = [ast.literal_eval(string_list) for string_list in df['tokens']]

In [13]:
sentiment_list= [ast.literal_eval(string_list) for string_list in df['word_sentiment']]

float

In [22]:
filtered_words_list = [[my_list[i][j] for j in range(len(sentiment_list[i])) if sentiment_list[i][j] == 1] for i in range(len(sentiment_list))]

In [26]:
filtered_words_list[0]

['delicious', 'food']

In [28]:
for i,a in enumerate(filtered_words_list): #fill in the empty lists
    if (len(a)==0):
        filtered_words_list[i].append(' ')

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
sentences = [' '.join(words) for words in filtered_words_list]
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(sentences)

In [41]:
hello=features.toarray()
hello=[torch.tensor(i) for i in hello]

In [44]:
len(hello[0])

541

In [49]:
sentiment_list=[torch.tensor(sent) for sent in df['sentiment']]

In [129]:
class MultiModel(nn.Module):
    def __init__(self):
        super(MultiModel, self).__init__()
        self.l1 = nn.Linear(541, 200)
        self.l2 = nn.Linear(200, 100)
        self.l3 = nn.Linear(100, 36)
        self.l4 = nn.Linear(36, 1)
        self.relu = nn.ReLU()
        self.tan=nn.Tanh()
        self.sig = nn.Sigmoid()
        
        
    def forward(self, features):
        hello = self.l1(features.to(torch.float32))
        hello = self.relu(hello)
        hello = self.l2(hello)
        hello = self.tan(hello)
        hello = self.l3(hello)
        hello = self.sig(hello)
        hello = self.l4(hello)
        return hello

In [130]:
from torch.utils.data import Dataset

class MultimodalDataset(Dataset):
    def __init__(self, texts, sentiments):
        self.texts = texts
        self.sentiments = sentiments

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        sentiment = self.sentiments[index]
        return (text, sentiment)

In [131]:
from sklearn.model_selection import train_test_split
# split the data into training and testing before being used
word_train, word_test,sentiments_train, sentiments_test = train_test_split(hello, sentiment_list, test_size=0.2)

In [132]:
from torch.utils.data import DataLoader

# create a dataset object and a dataloader object for the train set
train_dataset = MultimodalDataset(word_train, sentiments_train)
batch_size = 75
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# create a dataset object and a dataloader object for the test set
test_dataset = MultimodalDataset(word_test, sentiments_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [148]:
learning_rate = 0.001
batch_size = 32
epochs = 10
def train_loop(train_dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)
    for batch, (text_feats, sentiment_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(text_feats) 
        output = torch.sigmoid(output)
        loss = loss_fn(output.squeeze(), sentiment_labels.float())

        loss.backward()
        optimizer.step()
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(text_feats)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(test_dataloader, model, loss_fn):
    size = len(test_dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for text_feats, sentiment_labels in test_dataloader:
            output = model(text_feats)
            correct += (output.round() == sentiment_labels).type(torch.float).item()
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, For overall_sentiment")

In [149]:
from sklearn.metrics import precision_score, recall_score, f1_score

def test_loop2(test_dataloader, model, loss_fn):
    size = len(test_dataloader.dataset)
    model.eval()
    correct = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for text_feats, sentiment_labels in test_dataloader:
            output = model(text_feats)
            output = torch.sigmoid(output) 
            _, predicted = torch.max(output, dim=1)
            correct += (predicted == sentiment_labels).sum().item()
            
            y_true.extend(sentiment_labels.tolist())
            y_pred.extend(predicted.tolist())

    accuracy = correct / size
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

In [151]:
import torch.optim as optim

loss_fn = nn.BCELoss()
model=MultiModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop2(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.686058  [    0/31359]
loss: 0.664632  [  150/31359]
loss: 0.634395  [  300/31359]
loss: 0.532661  [  450/31359]
loss: 0.395107  [  600/31359]
loss: 0.230380  [  750/31359]
loss: 0.148934  [  900/31359]
loss: 0.099377  [ 1050/31359]
loss: 0.121738  [ 1200/31359]
loss: 0.059645  [ 1350/31359]
loss: 0.064799  [ 1500/31359]
loss: 0.032401  [ 1650/31359]
loss: 0.025179  [ 1800/31359]
loss: 0.137258  [ 1950/31359]
loss: 0.016936  [ 2100/31359]
loss: 0.015207  [ 2250/31359]
loss: 0.012884  [ 2400/31359]
loss: 0.075735  [ 2550/31359]
loss: 0.009936  [ 2700/31359]
loss: 0.008076  [ 2850/31359]
loss: 0.152119  [ 3000/31359]
loss: 0.041972  [ 3150/31359]
loss: 0.079195  [ 3300/31359]
loss: 0.008190  [ 3450/31359]
loss: 0.007267  [ 3600/31359]
loss: 0.118655  [ 3750/31359]
loss: 0.019378  [ 3900/31359]
loss: 0.056591  [ 4050/31359]
loss: 0.026303  [ 4200/31359]
loss: 0.055033  [ 4350/31359]
loss: 0.006608  [ 4500/31359]
loss: 0.006474  [ 4650/31359]


C:\Users\m_abd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 45.45%
Precision: 0.21
Recall: 0.45
F1 Score: 0.28
Epoch 2
-------------------------------
loss: 0.000812  [    0/31359]
loss: 0.000885  [  150/31359]
loss: 0.000990  [  300/31359]
loss: 0.000662  [  450/31359]
loss: 0.001177  [  600/31359]
loss: 0.001211  [  750/31359]
loss: 0.000654  [  900/31359]
loss: 0.000674  [ 1050/31359]
loss: 0.033671  [ 1200/31359]
loss: 0.029965  [ 1350/31359]
loss: 0.000715  [ 1500/31359]
loss: 0.000734  [ 1650/31359]
loss: 0.000801  [ 1800/31359]
loss: 0.005442  [ 1950/31359]
loss: 0.001033  [ 2100/31359]
loss: 0.001099  [ 2250/31359]
loss: 0.001110  [ 2400/31359]
loss: 0.018356  [ 2550/31359]
loss: 0.000949  [ 2700/31359]
loss: 0.000796  [ 2850/31359]
loss: 0.062008  [ 3000/31359]
loss: 0.000909  [ 3150/31359]
loss: 0.001048  [ 3300/31359]
loss: 0.001198  [ 3450/31359]
loss: 0.007108  [ 3600/31359]
loss: 0.001243  [ 3750/31359]
loss: 0.003395  [ 3900/31359]
loss: 0.001087  [ 4050/31359]
loss: 0.003349  [ 4200/31359]
loss: 0.004058  [ 4350/31359]

C:\Users\m_abd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 45.45%
Precision: 0.21
Recall: 0.45
F1 Score: 0.28
Epoch 3
-------------------------------
loss: 0.003236  [    0/31359]
loss: 0.003495  [  150/31359]
loss: 0.003483  [  300/31359]
loss: 0.000253  [  450/31359]
loss: 0.002973  [  600/31359]
loss: 0.002643  [  750/31359]
loss: 0.000226  [  900/31359]
loss: 0.000232  [ 1050/31359]
loss: 0.049085  [ 1200/31359]
loss: 0.003905  [ 1350/31359]
loss: 0.000286  [ 1500/31359]
loss: 0.000301  [ 1650/31359]
loss: 0.000316  [ 1800/31359]
loss: 0.012745  [ 1950/31359]
loss: 0.000333  [ 2100/31359]
loss: 0.000347  [ 2250/31359]
loss: 0.000331  [ 2400/31359]
loss: 0.018484  [ 2550/31359]
loss: 0.000310  [ 2700/31359]
loss: 0.000265  [ 2850/31359]
loss: 0.030371  [ 3000/31359]
loss: 0.000320  [ 3150/31359]
loss: 0.000317  [ 3300/31359]
loss: 0.000348  [ 3450/31359]
loss: 0.012254  [ 3600/31359]
loss: 0.000276  [ 3750/31359]
loss: 0.009505  [ 3900/31359]
loss: 0.000259  [ 4050/31359]
loss: 0.008016  [ 4200/31359]
loss: 0.007608  [ 4350/31359]

...

C:\Users\m_abd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 45.45%
Precision: 0.21
Recall: 0.45
F1 Score: 0.28
Epoch 9
-------------------------------
loss: 0.003637  [    0/31359]
loss: 0.003898  [  150/31359]
loss: 0.003846  [  300/31359]
loss: 0.000095  [  450/31359]
loss: 0.003244  [  600/31359]
loss: 0.002883  [  750/31359]
loss: 0.000054  [  900/31359]
loss: 0.000048  [ 1050/31359]
loss: 0.046186  [ 1200/31359]
loss: 0.003844  [ 1350/31359]
loss: 0.000080  [ 1500/31359]
loss: 0.000097  [ 1650/31359]
loss: 0.000110  [ 1800/31359]
loss: 0.013195  [ 1950/31359]
loss: 0.000090  [ 2100/31359]
loss: 0.000073  [ 2250/31359]
loss: 0.000059  [ 2400/31359]
loss: 0.017757  [ 2550/31359]
loss: 0.000033  [ 2700/31359]
loss: 0.000023  [ 2850/31359]
loss: 0.042674  [ 3000/31359]
loss: 0.000032  [ 3150/31359]
loss: 0.000043  [ 3300/31359]
loss: 0.000077  [ 3450/31359]
loss: 0.009846  [ 3600/31359]
loss: 0.000435  [ 3750/31359]
loss: 0.007338  [ 3900/31359]
loss: 0.000168  [ 4050/31359]
loss: 0.006046  [ 4200/31359]
loss: 0.005781  [ 4350/31359]

C:\Users\m_abd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 45.45%
Precision: 0.21
Recall: 0.45
F1 Score: 0.28
Epoch 10
-------------------------------
loss: 0.003732  [    0/31359]
loss: 0.004010  [  150/31359]
loss: 0.003950  [  300/31359]
loss: 0.000096  [  450/31359]
loss: 0.003311  [  600/31359]
loss: 0.002936  [  750/31359]
loss: 0.000045  [  900/31359]
loss: 0.000039  [ 1050/31359]
loss: 0.045728  [ 1200/31359]
loss: 0.003922  [ 1350/31359]
loss: 0.000058  [ 1500/31359]
loss: 0.000068  [ 1650/31359]
loss: 0.000080  [ 1800/31359]
loss: 0.012938  [ 1950/31359]
loss: 0.000071  [ 2100/31359]
loss: 0.000058  [ 2250/31359]
loss: 0.000047  [ 2400/31359]
loss: 0.017156  [ 2550/31359]
loss: 0.000026  [ 2700/31359]
loss: 0.000018  [ 2850/31359]
loss: 0.046957  [ 3000/31359]
loss: 0.000026  [ 3150/31359]
loss: 0.000037  [ 3300/31359]
loss: 0.000085  [ 3450/31359]
loss: 0.009213  [ 3600/31359]
loss: 0.000692  [ 3750/31359]
loss: 0.006709  [ 3900/31359]
loss: 0.000068  [ 4050/31359]
loss: 0.005630  [ 4200/31359]
loss: 0.005455  [ 4350/31359

C:\Users\m_abd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
